# Stock Price Service Testing (Using Cached Data)

This notebook tests the stock price system using cached data from the database,
avoiding Yahoo Finance API rate limits.

In [1]:
import sys

sys.path.append('..')

from datetime import UTC, datetime

import pandas as pd
from app.services.stock_price_service import stock_price_service
from sqlalchemy import func

from app.db.models import StockPrice
from app.db.session import SessionLocal

## View Cached Prices

Let's see what prices are already in the database.

In [2]:
db = SessionLocal()

try:
    # Get all cached prices
    prices = db.query(StockPrice).order_by(StockPrice.updated_at.desc()).limit(20).all()

    if not prices:
        print("No cached prices found in database.")
        print("You need to run the collection job first:")
        print("  make collect-stock-prices-top50")
    else:
        print(f"Found {len(prices)} cached prices:\n")

        data = []
        for sp in prices:
            age = datetime.now(UTC) - sp.updated_at
            minutes = int(age.total_seconds() / 60)
            hours = minutes // 60
            age_str = f"{hours}h {minutes % 60}m" if hours > 0 else f"{minutes}m"

            data.append({
                'Symbol': sp.symbol,
                'Price': f"${sp.price:.2f}",
                'Change': f"{sp.change_percent:+.2f}%" if sp.change_percent else '-',
                'Market': sp.market_state or '-',
                'Age': age_str,
                'Status': '✓ Fresh' if minutes < 30 else '⚠️  Stale'
            })

        df = pd.DataFrame(data)
        print(df.to_string(index=False))

finally:
    db.close()

Found 20 cached prices:

Symbol   Price  Change Market Age  Status
  HLLY   $2.98  -1.97%    PRE 13m ✓ Fresh
  MBWM  $45.74  +1.89%    PRE 13m ✓ Fresh
 MBVIU  $10.14  +0.10%    PRE 13m ✓ Fresh
  MPAA  $15.45  -5.96%    PRE 13m ✓ Fresh
   MPA  $11.26  -0.79%    PRE 13m ✓ Fresh
  MYCO  $24.86  -0.30%    PRE 13m ✓ Fresh
   ABP   $0.20  -6.14%    PRE 13m ✓ Fresh
  ABOT  $40.63  -0.11%    PRE 13m ✓ Fresh
  JIVE  $76.16  -0.14%    PRE 13m ✓ Fresh
  JIRE  $75.18  +0.11%    PRE 13m ✓ Fresh
  JILL  $15.73  -3.79%    PRE 13m ✓ Fresh
  HWKN $183.58  +0.38%    PRE 13m ✓ Fresh
  HWKE   $0.10       -    PRE 13m ✓ Fresh
   HWH   $2.83  +0.71%    PRE 13m ✓ Fresh
  NVBT  $36.25  +0.26%    PRE 13m ✓ Fresh
  NVAX   $9.08  -4.02%    PRE 13m ✓ Fresh
 NVAWW  $17.00 +28.65%    PRE 13m ✓ Fresh
 NVACW   $0.13  +8.33%    PRE 13m ✓ Fresh
  LCII  $90.19  -4.47%    PRE 14m ✓ Fresh
  LCID  $24.02  -3.03%    PRE 14m ✓ Fresh


## Test Cache Retrieval (No API Calls)

Test getting prices from cache without hitting the API.

In [3]:
db = SessionLocal()

try:
    # Get a symbol that exists in cache
    cached_symbol = db.query(StockPrice.symbol).first()

    if not cached_symbol:
        print("No cached data available. Run collection first.")
    else:
        symbol = cached_symbol[0]
        print(f"Testing cache retrieval for {symbol}...\n")

        # Get from cache (force_refresh=False, so it won't hit API if cache exists)
        price_data = await stock_price_service.get_or_refresh_price(db, symbol, force_refresh=False)

        if price_data:
            print("✓ Successfully retrieved from cache!\n")
            print(f"  Symbol: {price_data['symbol']}")
            print(f"  Price: ${price_data['price']}")
            print(f"  Change: ${price_data['change']} ({price_data['change_percent']}%)")
            print(f"  Market State: {price_data['market_state']}")
            print(f"  Last Updated: {price_data['last_updated']}")
        else:
            print("✗ Failed to retrieve from cache")

finally:
    db.close()

Testing cache retrieval for XT...



Yahoo Finance API error for XT: Too Many Requests. Rate limited. Try after a while.
Attempt 1 failed for XT, retrying in 1.0s
Yahoo Finance API error for XT: Too Many Requests. Rate limited. Try after a while.
Attempt 2 failed for XT, retrying in 2.0s
Yahoo Finance API error for XT: Too Many Requests. Rate limited. Try after a while.
Failed to fetch data for XT after 3 attempts
Invalid price data received for XT


✓ Successfully retrieved from cache!

  Symbol: XT
  Price: $73.96
  Change: $0.36 (0.49%)
  Market State: CLOSED
  Last Updated: 2025-10-03T16:49:49.604432+00:00


## Test Top N Ticker Selection

In [4]:
db = SessionLocal()

try:
    print("Getting top 10 most active tickers...\n")

    top_tickers = stock_price_service.get_top_n_tickers(db, n=10, hours=24)

    if top_tickers:
        print("Top 10 tickers (by 24h article count):\n")
        for i, symbol in enumerate(top_tickers, 1):
            # Get cached price if available
            sp = db.query(StockPrice).filter(StockPrice.symbol == symbol).first()
            price_str = f"${sp.price:.2f}" if sp else "No price data"
            print(f"  {i:2d}. {symbol:6s} - {price_str}")
    else:
        print("No active tickers found in last 24h")
        print("\nThis is normal if:")
        print("  - Database is new/empty")
        print("  - No Reddit scraping has run recently")

finally:
    db.close()

Getting top 10 most active tickers...

Top 10 tickers (by 24h article count):

   1. AMD    - $164.67
   2. HOOD   - $148.92
   3. NVDA   - $187.62
   4. RH     - $207.50
   5. SPY    - $669.21
   6. META   - $710.56
   7. RDDT   - $207.20
   8. APP    - $682.76
   9. ACHR   - $11.57
  10. TSLA   - $429.83


## Test Validation Logic

This doesn't require API access.

In [ ]:
import math

test_cases = [
    ({"symbol": "TEST", "price": 150.50}, True, "Valid price"),
    ({"symbol": "TEST", "price": None}, False, "None price"),
    ({"symbol": "TEST", "price": math.nan}, False, "NaN price"),
    ({"symbol": "TEST", "price": 0.0}, False, "Zero price"),
    ({"symbol": "TEST", "price": -10.0}, False, "Negative price"),
    ({"symbol": "TEST", "price": 0.01}, True, "Penny stock"),
]

print("Testing validation logic (no API required):\n")
print("=" * 60)

for data, expected, description in test_cases:
    result = stock_price_service.validate_price_data(data)
    status = "✓" if result == expected else "✗"
    result_str = "Valid" if result else "Invalid"
    print(f"{status} {description:30s} -> {result_str}")

print("=" * 60)

## Database Statistics

In [ ]:
db = SessionLocal()

try:
    total_count = db.query(func.count(StockPrice.symbol)).scalar() or 0
    stale_count = stock_price_service.get_stale_price_count(db)
    fresh_count = total_count - stale_count

    print("Database Statistics:\n")
    print(f"  Total prices: {total_count}")
    print(f"  Fresh (<30 min): {fresh_count}")
    print(f"  Stale (>30 min): {stale_count}")

    if total_count > 0:
        freshness_pct = fresh_count / total_count * 100
        print(f"\n  Freshness: {freshness_pct:.1f}%")

        if freshness_pct >= 90:
            print("  ✓ Excellent!")
        elif freshness_pct >= 70:
            print("  ✓ Good")
        else:
            print("  ⚠️  Consider running: make collect-stock-prices-top50")

finally:
    db.close()

## How to Get Fresh Data

To populate the cache with fresh data, run from terminal:

```bash
# Wait a few hours for rate limits to clear, then run:
make collect-stock-prices-top50
```

Or use a different time:
- Early morning (before market open)
- Late evening
- Different network/VPN